# Naïve Bayes

What a fun name for a classifier, don't you think? :)

Naïve Bayes, simply put, refer to a family of algorithms that apply the Bayes' theorem with the _naïve_ assumption that all features are independent from each other (Hence the name). 

Although there are (very) rare situations where all features are independent, most of the time there are at least some degree of correlation between two or more of them. For instance, one could infer that an object that is a fruit, is red, has a somewhat round shape and grows in trees is most likely an apple. Why? Because each feature gives us more information about the others and that increases our confidence in the output. Nonetheless, Naïve Bayes algorithm make the strong assumption that in any instance this correlation occurs.

It doesn't sound like an accurate algorithm, right? Well, the thing is that in real life, despite this naïvety, it works really well and it also facilitates the computation __a lot__.

Bayes' Theorem gives us a way to calculate the probability of a piece of data belonging to a particular class, given our prior knowledge.

$$ P(class|data)=\frac{P(data|class)*P(class)}{P(data)} $$

It is somewhat confusing, but what this formula says is "The posterior probability of the class given the data is defined as the prior probability of that data given the class by the likelihood of the class divided by the evidence of the data". So much clearer now! (Fingers crossed)

A nicer way to represent this formula is:

$$ posterior=\frac{prior*likelihood}{evidence} $$

Let's start our implementation by loading the code and libraries we'll need. We will build our solution on top of the ones we implemented in the [previous notebook](https://github.com/jesus-a-martinez-v/toy-ml/blob/master/src/main/scala/notebooks/lperceptron.ipynb).

In [1]:
import $ivy.`com.github.tototoshi::scala-csv:1.3.5`
import $file.^.datasmarts.ml.toy.scripts.Perceptron, Perceptron._
import scala.util.Random

import $ivy.$                                      

import $file.$                                     , Perceptron._

import scala.util.Random

## Data

We'll use the [Iris Flower](http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) dataset. It involves the prediction of flower species given measurements of iris flowers. 

Let's load the data:

In [2]:
val BASE_DATA_PATH = "../../resources/data"
val irisPath = s"$BASE_DATA_PATH/12/iris.csv"

val rawData = loadCsv(irisPath)
val numberOfRows = rawData.length
val numberOfColumns = rawData.head.length
println(s"Number of rows in dataset: $numberOfRows")
println(s"Number of columns in dataset: $numberOfColumns")

val (data, lookUpTable) = {
    val dataWithNumericColumns = (0 until (numberOfColumns - 1)).toVector.foldLeft(rawData) { (d, i) => textColumnToNumeric(d, i)}
    categoricalColumnToNumeric(dataWithNumericColumns, numberOfColumns - 1)
}

Number of rows in dataset: 150
Number of columns in dataset: 5


BASE_DATA_PATH: String = "../../resources/data"
irisPath: String = "../../resources/data/12/iris.csv"
rawData: Vector[Vector[Data]] = Vector(
  Vector(Text(5.1), Text(3.5), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.9), Text(3.0), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.7), Text(3.2), Text(1.3), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.6), Text(3.1), Text(1.5), Text(0.2), Text(Iris-setosa)),
  Vector(Text(5.0), Text(3.6), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(5.4), Text(3.9), Text(1.7), Text(0.4), Text(Iris-setosa)),
  Vector(Text(4.6), Text(3.4), Text(1.4), Text(0.3), Text(Iris-setosa)),
  Vector(Text(5.0), Text(3.4), Text(1.5), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.4), Text(2.9), Text(1.4), Text(0.2), Text(Iris-setosa)),
  Vector(Text(4.9), Text(3.1), Text(1.5), Text(0.1), Text(Iris-setosa)),
  Vector(Text(5.4), Text(3.7), Text(1.5), Text(0.2), Text(Iris-setosa)),
...
numberOfRows: Int = 150
numberOfColumns: Int = 5
da

## Separation by Class

In a moment we will need to calculate the probability  of data by the class they belong to. In order to do that, we'll need to first searate our training set by classes. Fairly easy:

In [3]:
def separateByClass(dataset: Dataset): Map[Data, Vector[Vector[Data]]] = {
  dataset.groupBy(_.last)
}

defined function separateByClass

Let's test it on a mock dataset:

In [4]:
val mockDataset = Vector(
  (3.393533211, 2.331273381, 0),
  (3.110073483, 1.781539638, 0),
  (1.343808831, 3.368360954, 0),
  (3.582294042, 4.67917911, 0),
  (2.280362439, 2.866990263, 0),
  (7.42346942, 4.696522875, 1),
  (5.745051997, 3.533989803, 1),
  (9.172168622, 2.511101045, 1),
  (7.7922783481, 3.424088941, 1),
  (7.939820817, 0.791637231, 1)
) map { case (x1, x2, y) => Vector(Numeric(x1), Numeric(x2), Numeric(y))}

val separated = separateByClass(mockDataset)

mockDataset: Vector[Vector[Numeric]] = Vector(
  Vector(Numeric(3.393533211), Numeric(2.331273381), Numeric(0.0)),
  Vector(Numeric(3.110073483), Numeric(1.781539638), Numeric(0.0)),
  Vector(Numeric(1.343808831), Numeric(3.368360954), Numeric(0.0)),
  Vector(Numeric(3.582294042), Numeric(4.67917911), Numeric(0.0)),
  Vector(Numeric(2.280362439), Numeric(2.866990263), Numeric(0.0)),
  Vector(Numeric(7.42346942), Numeric(4.696522875), Numeric(1.0)),
  Vector(Numeric(5.745051997), Numeric(3.533989803), Numeric(1.0)),
  Vector(Numeric(9.172168622), Numeric(2.511101045), Numeric(1.0)),
  Vector(Numeric(7.7922783481), Numeric(3.424088941), Numeric(1.0)),
  Vector(Numeric(7.939820817), Numeric(0.791637231), Numeric(1.0))
)
separated: Map[Data, Vector[Vector[Data]]] = Map(
  Numeric(1.0) -> Vector(
    Vector(Numeric(7.42346942), Numeric(4.696522875), Numeric(1.0)),
    Vector(Numeric(5.745051997), Numeric(3.533989803), Numeric(1.0)),
    Vector(Numeric(9.172168622), Numeric(2.511101045), Num

Good. We can see it is working as expected by inspecting the rows that form each group.

## Summarize Dataset

The next step is to obtain two very important statistics for each feature (column) in the dataset:

 - Mean.
 - Standard Deviation.
 
In order to be a bit more efficient, we'll collect each of these statistics along with the row count per feature in one pass:

In [5]:
def summarizeDataset(dataset: Dataset) = {
  val numberOfColumns = dataset.head.length

  val means = getColumnsMeans(dataset)
  val standardDeviations = getColumnsStandardDeviations(dataset, means)
  val counts = (1 to dataset.head.length).toVector.map(_ => dataset.length)

  assert(List(means.length, standardDeviations.length, counts.length).forall(_ == numberOfColumns))

  // We ignore the labels column
  (0 until numberOfColumns - 1).toVector.map(i => (means(i).get, standardDeviations(i).get, counts(i)))
}

defined function summarizeDataset

Let's test it:

In [6]:
summarizeDataset(mockDataset)

res5: Vector[(Double, Double, Int)] = Vector(
  (5.178286121009999, 2.766534398702562, 10),
  (2.9984683241, 1.218556343617447, 10)
)

The first triplet corresponds to the statistics of X1 and the second to the statistics of X2. In a table:

|                    | X1                | X2                |
|--------------------|-------------------|-------------------|
| __Mean__               | 5.178286121009999 | 2.9984683241      |
| __Standard Deviation__ | 2.766534398702562 | 1.218556343617447 |
| __Count__              | 10                | 10                |

## Summarize Data by Class

With these two functions implemented we have all we need to summarize each subset of data corresponding to each class. We'll keep each summary in a map where the label is the class and the value is a list of triplets (mean, standard deviation and count) per feature.

In [7]:
def summarizeByClass(dataset: Dataset) = {
  val separated = separateByClass(dataset)

  separated.mapValues(summarizeDataset)
}

defined function summarizeByClass

Let's test it:

In [8]:
val summaries = summarizeByClass(mockDataset)

summaries: Map[Data, Vector[(Double, Double, Int)]] = Map(
  Numeric(1.0) -> Vector(
    (7.61455784082, 1.2344126956449888, 5),
    (2.9914679790000003, 1.4541931384601618, 5)
  ),
  Numeric(0.0) -> Vector(
    (2.7420144012, 0.9265683289298018, 5),
    (3.0054686692, 1.1073295894898725, 5)
  )
)

## Gaussian Probability Density Function

Calculating the probability of observing a given value is quite hard. One of the tricks used to do it is to _assume_ that this value is drawn from a distribution (in this case, a Gaussian distribution).

The good thing about Gaussian distributions is that they can be summarized with only two numbers: Mean and standard deviation. With the mighty power of math we can _estimate_ the probability of some value _X_:

$$ P(X)=\frac{1}{\sqrt{2*\pi}*\sigma}e^{-(\frac{(X - \mu)^2}{(2*\sigma)^2})}$$

Let's create a function that does the heavy lifting for us:

In [9]:
def calculateProbability(x: Double, mean: Double, standardDeviation: Double) = {
  val exponent = math.exp(-(math.pow(x - mean, 2) / (2 * standardDeviation * standardDeviation)))
  (1.0 / (math.sqrt(2 * math.Pi) * standardDeviation)) * exponent
}

calculateProbability(1.0, 1.0, 1.0)
calculateProbability(2.0, 1.0, 1.0)
calculateProbability(0.0, 1.0, 1.0)

defined function calculateProbability
res8_1: Double = 0.3989422804014327
res8_2: Double = 0.24197072451914337
res8_3: Double = 0.24197072451914337

## Class Probabilities

Now we need to compute the class probabilities for new data given the statistics summary per feature. These are calculated separately for each class in the dataset.

The probability that some value corresponds to a class is:

$$ P(class|data)=P(X|data)*P(class)$$

This is not exactly the Bayes Theorem. What does dropping the denominator causes? Well, $P(data)$ is actually a normalization factor that makes sure that the value we are calculating falls in the range [0, 1] and, hence, can be considered an actual probability. Given that we are more interested in the classification task rather than the probability itself, we can save ourselves computation resources and still be sure we'll classify each data point right.

In [10]:
def calculateClassProbabilities(summaries: Map[Data, Vector[(Double, Double, Int)]], row: Vector[Data]) = {
  val totalRows = summaries.foldLeft(0){ (accum, entry) =>
    entry match {
      case (_, summary) => accum + summary.head._3
    }
  }

  summaries.mapValues { summaries =>
    var a = summaries.head._3 / totalRows.toDouble

    // Class probability
    summaries.indices.foldLeft(summaries.head._3 / totalRows.toDouble) { (classProbability, i) =>
      val (mean, standardDeviation, _) = summaries(i)
      classProbability * calculateProbability(getNumericValue(row(i)).get, mean, standardDeviation)
    }
  }
}

defined function calculateClassProbabilities

In [11]:
val probabilities = calculateClassProbabilities(summaries, mockDataset.head)

probabilities: Map[Data, Double] = Map(Numeric(1.0) -> 1.1558796759299434E-4, Numeric(0.0) -> 0.05032427673372076)

We see that the probability of the first row belonging to the first class (0) is quite higher that the probability it belongs to the second class (1).

## Naïve Bayes

We can now make predictions using all the functions we've implemented so far. Let's see:

In [12]:
def predictNB(summaries: Map[Data, Vector[(Double, Double, Int)]], row: Vector[Data]): Data = {
  val probabilities = calculateClassProbabilities(summaries, row)

  val (Some(bestLabel), _) = probabilities.foldLeft((None: Option[Data], -1.0)) { (bestLabelAndProb, entry) =>
      entry match {
        case (label, classProbability) =>
          val (bestLabel, bestProbability) = bestLabelAndProb

          if (bestLabel.isEmpty || classProbability > bestProbability) {
            (Some(label), classProbability)
          } else {
            bestLabelAndProb
          }
      }
  }

  bestLabel
}


defined function predictNB

In [13]:
def naiveBayes(train: Dataset, test: Dataset) = {
  val summaries = summarizeByClass(train)

  test.map { row =>
    predictNB(summaries, row)
  }
}

defined function naiveBayes

Good.

Let's now use our new algorithm to test it on the Iris dataset.

We'll start by running a baseline model on it and then our freshly implemented Gaussian Naïve Bayes algorithm and then we will compare their performance.

As a baseline we will use a __zero rule classifier__.

In [14]:
// Normalize data
val minMax = getDatasetMinAndMax(data)
val normalizedData = normalizeDataset(data, minMax)

val baselineAccuracy = evaluateAlgorithmUsingTrainTestSplit[Numeric](
        normalizedData, 
        (train, test, parameters) => zeroRuleClassifier(train, test), 
        Map.empty, 
        accuracy, 
        trainProportion=0.8)

println(s"Zero Rule Algorithm accuracy: $baselineAccuracy")

Zero Rule Algorithm accuracy: 0.23333333333333334


minMax: MinMaxData = Vector(
  Some((4.3, 7.9)),
  Some((2.0, 4.4)),
  Some((1.0, 6.9)),
  Some((0.1, 2.5)),
  Some((0.0, 2.0))
)
normalizedData: Dataset = Vector(
  Vector(
    Numeric(0.22222222222222213),
    Numeric(0.6249999999999999),
    Numeric(0.06779661016949151),
    Numeric(0.04166666666666667),
    Numeric(0.0)
  ),
  Vector(
    Numeric(0.1666666666666668),
    Numeric(0.41666666666666663),
    Numeric(0.06779661016949151),
...
baselineAccuracy: Double = 0.23333333333333334

In [15]:
val naiveBayesAccuracy = evaluateAlgorithmUsingTrainTestSplit[Numeric](
    normalizedData,
    (train, test, parameters) => naiveBayes(train, test),
    Map.empty,
    accuracy,
    trainProportion=0.8)

println(s"Naive Bayes accuracy: $naiveBayesAccuracy")

Naive Bayes accuracy: 0.9333333333333333


naiveBayesAccuracy: Double = 0.9333333333333333

Wow! It's evident that the 93.334% accuracy achieved by our Gaussian Naïve Bayes is dramatically better that the ine achieved by the baseline model (23.334%)